In [1]:
import sys
sys.path.append("..")

In [2]:
from keras.models import Model

Using TensorFlow backend.


In [3]:
from sklearn.datasets import fetch_20newsgroups
from pprint import pprint
import numpy as np
from numpy.linalg import norm
import pickle

In [4]:
from hyper_params import *
import text_encoder as te
import text_decoder as td
from data_set import *

In [5]:
import bowizer
import tfidf

In [6]:
with open('../data/norvig/big.txt') as f:
    norvig = f.read()

In [7]:
chunks = make_chunks(norvig)

In [8]:
r = np.random.RandomState(42)

In [9]:
r.shuffle(chunks)
train_text, test_text = make_train_test(chunks)

In [10]:
tm = bowizer.TokenMaker([norvig], 3000)

In [11]:
embed_h = EmbeddingHyper(tm.vocab_size + 1, 64)
conv_h = ConvHyper(128, 6, 4)
rnn_h = RnnHyper(256, is_lstm=False, is_bidirectional=True, return_sequences=False)
encoder_h = te.Hyper(embed_h, [conv_h, rnn_h])

In [12]:
dernn_h = RnnHyper(64, is_lstm=False, is_bidirectional=False, return_sequences=True, unroll=True)
dec_h = DeconvHyper(128, 6, 4)
decoder_h = td.Hyper(tm.vocab_size + 1, [dernn_h, dec_h])

In [13]:
encoder = encoder_h.make_layer()
decoder = decoder_h.make_layer()

In [14]:
def make_model(max_len):
    x = Input(shape=(max_len,), name='text_input')
    h = encoder(x)
    h = decoder(h, max_len)
    model = Model(x, h)
    model.compile(optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['categorical_accuracy'])

    return model

In [15]:
model64, model128, model256 = make_model(64), make_model(128), make_model(256)

In [16]:
models = {64: model64, 128: model128, 256: model256 }

In [23]:
def training_round(max_len, history=None):
    if history is None:
        initial_epoch = 0
    else:
        initial_epoch = len(history['loss'])
    train, test = bowizer.SlicedWordData.Random(train_text, test_text, 1000, max_len, 10000, r, tm)
    model = models[max_len]
    newhistory = model.fit(x=train.x, y=train.y,
                            epochs=initial_epoch+5, batch_size=16,
                            validation_data=(test.x, test.y),
                            initial_epoch=initial_epoch)
    if history is None:
        history = newhistory.history
    else:
        history = {key:history[key] + newhistory.history[key] for key in history.keys()}
    return history

In [24]:
history = None

In [25]:
for i in range(2):
    print(i)
    history = training_round(64, history)

0
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 72s - loss: 5.0114 - categorical_accuracy: 0.2158 - val_loss: 4.8416 - val_categorical_accuracy: 0.2610
Epoch 2/5
10000/10000 [==============================] - 67s - loss: 4.9509 - categorical_accuracy: 0.2158 - val_loss: 4.6726 - val_categorical_accuracy: 0.2610
Epoch 3/5
10000/10000 [==============================] - 70s - loss: 4.8280 - categorical_accuracy: 0.2160 - val_loss: 4.5890 - val_categorical_accuracy: 0.2615
Epoch 4/5
10000/10000 [==============================] - 70s - loss: 4.7587 - categorical_accuracy: 0.2165 - val_loss: 4.5583 - val_categorical_accuracy: 0.2618
Epoch 5/5
10000/10000 [==============================] - 70s - loss: 4.7222 - categorical_accuracy: 0.2166 - val_loss: 4.5286 - val_categorical_accuracy: 0.2614
1
Train on 10000 samples, validate on 1000 samples
Epoch 6/10
10000/10000 [==============================] - 73s - loss: 4.6894 - categorical_ac